# This project aims to predict the screentime 

In [49]:
import numpy as np
from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## Loading the data

In [50]:
raw_data = pd.read_csv('user_behavior_dataset.csv')
raw_data.head

<bound method NDFrame.head of      User ID        Device Model Operating System  App Usage Time (min/day)  \
0          1      Google Pixel 5          Android                       393   
1          2           OnePlus 9          Android                       268   
2          3        Xiaomi Mi 11          Android                       154   
3          4      Google Pixel 5          Android                       239   
4          5           iPhone 12              iOS                       187   
..       ...                 ...              ...                       ...   
695      696           iPhone 12              iOS                        92   
696      697        Xiaomi Mi 11          Android                       316   
697      698      Google Pixel 5          Android                        99   
698      699  Samsung Galaxy S21          Android                        62   
699      700           OnePlus 9          Android                       212   

     Screen On Time (

In [51]:
raw_data.describe(include='all')

,User ID,Device Model,Operating System,App Usage Time (min/day),Screen On Time (hours/day),Battery Drain (mAh/day),Number of Apps Installed,Data Usage (MB/day),Age,Gender,User Behavior Class
count,700.00000,700,700,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700,700.000000
unique,NaN,5,2,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
top,NaN,Xiaomi Mi 11,Android,NaN,NaN,NaN,NaN,NaN,NaN,Male,NaN
freq,NaN,146,554,NaN,NaN,NaN,NaN,NaN,NaN,364,NaN
mean,350.50000,NaN,NaN,271.128571,5.272714,1525.158571,50.681429,929.742857,38.482857,NaN,2.990000
std,202.21688,NaN,NaN,177.199484,3.068584,819.136414,26.943324,640.451729,12.012916,NaN,1.401476
min,1.00000,NaN,NaN,30.000000,1.000000,302.000000,10.000000,102.000000,18.000000,NaN,1.000000
25%,175.75000,NaN,NaN,113.250000,2.500000,722.250000,26.000000,373.000000,28.000000,NaN,2.000000
50%,350.50000,NaN,NaN,227.500000,4.900000,1502.500000,49.000000,823.500000,38.000000,NaN,3.000000
75%,525.25000,NaN,NaN,434.250000,7.400000,2229.500000,74.000000,1341.000000,49.000000,NaN,4.000000


In [52]:
raw_data = raw_data.dropna(axis=0, how='any')

In [53]:
raw_data = raw_data.drop('Device Model',axis=1)
raw_data = raw_data.drop('User ID',axis=1)
raw_data.head()

,Operating System,App Usage Time (min/day),Screen On Time (hours/day),Battery Drain (mAh/day),Number of Apps Installed,Data Usage (MB/day),Age,Gender,User Behavior Class
0,Android,393,6.4,1872,67,1122,40,Male,4
1,Android,268,4.7,1331,42,944,47,Female,3
2,Android,154,4.0,761,32,322,42,Male,2
3,Android,239,4.8,1676,56,871,20,Male,3
4,iOS,187,4.3,1367,58,988,31,Female,3


In [54]:
raw_data['Operating System'] = raw_data['Operating System'].map({'Android': 0, 'iOS': 1})
raw_data['Gender'] = raw_data['Gender'].map({'Male': 0, 'Female': 1})
raw_data    

,Operating System,App Usage Time (min/day),Screen On Time (hours/day),Battery Drain (mAh/day),Number of Apps Installed,Data Usage (MB/day),Age,Gender,User Behavior Class
0,0,393,6.4,1872,67,1122,40,0,4
1,0,268,4.7,1331,42,944,47,1,3
2,0,154,4.0,761,32,322,42,0,2
3,0,239,4.8,1676,56,871,20,0,3
4,1,187,4.3,1367,58,988,31,1,3
...,...,...,...,...,...,...,...,...,...
695,1,92,3.9,1082,26,381,22,0,2
696,0,316,6.8,1965,68,1201,59,0,4
697,0,99,3.1,942,22,457,50,1,2
698,0,62,1.7,431,13,224,44,0,1


In [55]:
col_to_move = raw_data.columns[2]  # Get column name at index 1
new_order = [col for col in raw_data.columns if col != col_to_move] + [col_to_move]

# Rearranging
raw_data = raw_data[new_order]
raw_data

,Operating System,App Usage Time (min/day),Battery Drain (mAh/day),Number of Apps Installed,Data Usage (MB/day),Age,Gender,User Behavior Class,Screen On Time (hours/day)
0,0,393,1872,67,1122,40,0,4,6.4
1,0,268,1331,42,944,47,1,3,4.7
2,0,154,761,32,322,42,0,2,4.0
3,0,239,1676,56,871,20,0,3,4.8
4,1,187,1367,58,988,31,1,3,4.3
...,...,...,...,...,...,...,...,...,...
695,1,92,1082,26,381,22,0,2,3.9
696,0,316,1965,68,1201,59,0,4,6.8
697,0,99,942,22,457,50,1,2,3.1
698,0,62,431,13,224,44,0,1,1.7


## Converting hours to minutes:


In [56]:
raw_data['Screen On Time (minutes/day)'] = raw_data['Screen On Time (hours/day)'] * 60
raw_data

C:\Users\a_no7\AppData\Local\Temp\ipykernel_20988\1634411379.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data['Screen On Time (minutes/day)'] = raw_data['Screen On Time (hours/day)'] * 60


,Operating System,App Usage Time (min/day),Battery Drain (mAh/day),Number of Apps Installed,Data Usage (MB/day),Age,Gender,User Behavior Class,Screen On Time (hours/day),Screen On Time (minutes/day)
0,0,393,1872,67,1122,40,0,4,6.4,384.0
1,0,268,1331,42,944,47,1,3,4.7,282.0
2,0,154,761,32,322,42,0,2,4.0,240.0
3,0,239,1676,56,871,20,0,3,4.8,288.0
4,1,187,1367,58,988,31,1,3,4.3,258.0
...,...,...,...,...,...,...,...,...,...,...
695,1,92,1082,26,381,22,0,2,3.9,234.0
696,0,316,1965,68,1201,59,0,4,6.8,408.0
697,0,99,942,22,457,50,1,2,3.1,186.0
698,0,62,431,13,224,44,0,1,1.7,102.0


## Saving it to a file 
Create another file without the headers

In [57]:
raw_data.to_csv('preprocessed.csv', header=False, index=False)
raw_csv_data = np.loadtxt('preprocessed.csv', delimiter=',') 
raw_csv_data

array([[0.000e+00, 3.930e+02, 1.872e+03, ..., 4.000e+00, 6.400e+00,
        3.840e+02],
       [0.000e+00, 2.680e+02, 1.331e+03, ..., 3.000e+00, 4.700e+00,
        2.820e+02],
       [0.000e+00, 1.540e+02, 7.610e+02, ..., 2.000e+00, 4.000e+00,
        2.400e+02],
       ...,
       [0.000e+00, 9.900e+01, 9.420e+02, ..., 2.000e+00, 3.100e+00,
        1.860e+02],
       [0.000e+00, 6.200e+01, 4.310e+02, ..., 1.000e+00, 1.700e+00,
        1.020e+02],
       [0.000e+00, 2.120e+02, 1.306e+03, ..., 3.000e+00, 5.400e+00,
        3.240e+02]])

## Mapping inputs and outputs:

In [64]:
unscaled_inputs_all = raw_csv_data[:,:-1]
targets_all = raw_csv_data[:,-1]

array([384., 282., 240., 288., 258., 120., 438., 684., 462., 396.,  84.,
       330., 372., 294., 324., 624., 216., 348.,  84., 510., 156., 696.,
       390., 336., 240., 204., 456., 630., 408., 372., 360., 276., 282.,
       120., 612.,  72., 198., 672., 384., 372., 282., 606., 126., 222.,
       144., 654., 504., 216., 492., 522., 444., 354., 162., 132., 522.,
        96.,  66., 504., 420., 426., 714., 450., 114., 690., 420., 240.,
       270., 240., 612., 102., 432., 540., 378.,  66., 120., 342., 120.,
       150., 108., 342., 636., 168., 432., 108., 270., 444., 144.,  90.,
       198., 372., 126., 246.,  84., 150., 450., 432., 570., 720.,  72.,
       330.,  66., 246., 150., 252., 168.,  72., 162.,  84., 330., 360.,
       228., 516., 306., 192., 474., 192.,  72., 312.,  96., 162., 396.,
       522., 600., 486., 306., 708., 444., 252., 150., 198., 648., 432.,
       216.,  60.,  96., 678., 102., 270., 582., 666., 312., 378., 132.,
        66., 672., 318., 444.,  96., 252., 222., 56

## Scaling the inputs

In [59]:
scaled_inputs = preprocessing.scale(unscaled_inputs_all)

## Shuffling

In [60]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_all[shuffled_indices]

## 80-10-10 Train Validation Test split

In [61]:
sample_count = shuffled_targets.shape[0]

train_samples_count = int(sample_count * 0.8)
validation_samples_count = int(sample_count * 0.1)
test_samples_count = int(sample_count * 0.1)

# training data
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# validation data
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

# testing data
test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

# Checking
print(np.shape(train_inputs))
print(np.shape(validation_inputs))
print(np.shape(test_inputs))

(560, 9)
(70, 9)
(70, 9)


In [62]:
np.savez('Screentime_train', inputs = train_inputs, targets = train_targets)
np.savez('Screentime_validation', inputs = validation_inputs, targets = validation_targets)
np.savez('Screentime_test', inputs = test_inputs, targets = test_targets)